# Distance Calculation

##Euclidian Distance

$$\|P\|= n \in P_{n}\sqrt{p_{n1}^2+p_{n2}^2+\cdots +p_{n\dotso}^2} = \sqrt{p \cdot p}$$

###Calculating with one vector:
$$\langle a, b, c \rangle \to \sqrt{a^2 + b^2 + c^2}$$

$$\langle 3, 4, 5 \rangle \to \sqrt{3^2 + 4^2 + 5^2}$$

###Calculating with two vectors:
$${\langle a_{x}, a_{y}, a_{z} \rangle, \langle b_{x}, b_{y}, b_{z} \rangle} \to \sqrt{(a_{x}-b_{x})^2 + (a_{y}-b_{y})^2 + (a_{z}-b_{z})^2} $$
$$\langle 2, 5, 18 \rangle, \langle 7, 31, 43\rangle \to \sqrt{(2-7)^2 + (5-31)^2 + (18 - 43)^2}$$

Because you square the differences between the vectors, you always end up with positive distances.

In [1]:
from math import sqrt
import numpy as np
from __future__ import division
import nltk

In [4]:
### Example Euclidian distance without numpy

def euclidian_non_np(vector_a, vector_b=None): # We assume that we have only one vector.
    if vector_b:  # we have two vectors == if vector_b != None
        # zip merges two lists, like a zipper on your coat
        distance_sums = sum((a - b)**2 for a,b in zip(vector_a, vector_b))  
        
        return sqrt(distance_sums)
    
    return sqrt(sum(a**2 for a in vector_a))

## example Euclidian distance with one or two vectors using numpy

def euclidian_np(a, b=None): # We assume that we have only one vector.
    if b:  # we have two vectors
        # function ends and return the two vectors
        # with numpy arrays we can use minus operator to 
        # calculate the difference between each element.
        return np.linalg.norm(np.array(a) - np.array(b)) 
    
    return np.linalg.norm(a)  # we have only one vector

print('Non numpy distance [3,4,5]          :', euclidian_non_np([3,4,5]))
print('Non numpy distance [3,4,5], [4,5,6] :', euclidian_non_np([3,4,5],[4,5,6]))
print('With numpy distance [3,4,5]         :', euclidian_np([3,4,5]))
print('With numpy distance [3,4,5], [4,5,6]:', euclidian_np([3,4,5],[4,5,6]))

Non numpy distance [3,4,5]          : 7.0710678118654755
Non numpy distance [3,4,5], [4,5,6] : 1.7320508075688772
With numpy distance [3,4,5]         : 7.07106781187
With numpy distance [3,4,5], [4,5,6]: 1.73205080757


# Similarity Calculation
## Cosine Similarity

$$similarity(A, B) = \cos() = \frac{A \cdot B}{\|A\| * \|B\|}$$

Calculating the numerator is the dot product of the two vectors. This is the same as the sum of the pairwise product of the elements in the vectors.

$$
\langle 2, 5, 18 \rangle
\langle 7, 31, 43 \rangle \to 2 \cdot 7 + 5 \cdot 31 + 18 \cdot 43
$$

The denominator is the product of the two Euclidian distances from both vectors.

$$\langle 2, 5, 18 \rangle, \langle 7, 31, 43\rangle \to \sqrt{(2)^2 + (5)^2 + (18)^2} * \sqrt{(7)^2 + (31)^2 + (43)^2}$$

Which results in:
$$\frac{2 \cdot 7 + 5 \cdot 31 + 18 \cdot 43}{\sqrt{(2)^2 + (5)^2 + (18)^2} * \sqrt{(7)^2 + (31)^2 + (43)^2}}$$

In [5]:
def similarity(vector_a, vector_b):
    teller = np.vdot(np.array(vector_a), np.array(vector_b)) # dot product of two vectors
    noemer = euclidian_np(vector_a, vector_b)  # reusing the euclidian distance
    return teller / noemer

#Intrinsic en extrinsic search methods

##Intrinsic
###TF * IDF
Intrinsic methods of defining the worth of a document in terms of the query proposed to it is based around the idea that the context surrounding the document is not necessary to say something about the value of the document. This approach regards every document as a set of words with their frequency connected to it and the collection of documents as a set of the aforementioned sets.

As an example you could take a collection of news articles, or all a collection of pages from Wikipedia.

A method of defining the value of a document with regards to a search query Q is by multiplying the amount of times a word appears in a document (TF) with the inverse document frequency (IDF).

$$TF = \textit{amount of times a word appears in a document.}\\
N = \textit{amount of documents}\\
n = \textit{amount of document the word appears in}\\
IDF = \log{N/n}$$

###Indexing
Recalculating the TF \* IDF by processing every file when a query is very time consuming as the collection of documents grows. Therefore it is a much better solution to store an index of the collection containing the information needed to calculate the TF \* IDF. Re-indexing is necessary over time, and we want to extract as much information as possible from the indexing as this will reduce the calculationss necessary at query time. Below is a sample index layout based on a python dictionary:

```
index = { __filenames: [list of filenames or document names],
                  __N: number of documents in the collection,
                word: { doc1 : TF,
                         doc2 : TF,
                         doc3 : TF,
                           DF : N / number of documents the word1 appears in},
                word2: { doc1 : TF,
                         doc2 : TF,
                         doc3 : TF,
                           DF : N / number of documents the word2 appears in}
        }
```
In a real world example the index would of course be stored in a database to provide stability and redundance. Also RAM size is a limitation in terms of index size. 

Below is a sample implementation that will read any text file in a given folder.

In [6]:
def index_collection(folder):
    '''
    Returns an index dictionary of all tokens in all ".txt" files
    in the given folder.
    >>index_collection('my_folder') 
    ''' 
    index = {}
    folder = os.path.abspath(folder)
    
    # check for a trailing slash and add it if it is not there
    if folder.endswith() != '/':
        folder = folder + '/'
        
    # create list of file names and only use the files ending on .txt 
    # add the absolute path to the filename so we can use the index everywhere
    files = [folder + f for f in os.listdir() if '.txt' in f] 
    # setting up the index
    
    N = len(files)
    index['__filenames'] = files 
    index['__N'] = N
    
    stop_words = nltk.corpus.stopwords.words('english')
    
    # loop over every file to analyze the document.
    for filename in files:
        with open(filename) as f:
            text = f.read().lower()  # read the whole file and lowercase it.
            tokens = [word for word in nltk.tokenize.word_tokenize(text)
                         if word not in stop_words]
            fd = nltk.FreqDist(tokens)  # Generate a frequency distribution
                                        # {word: frequency}
        # loop over every word
        for word, freq in fd.items():
            idx_word = index.get(word)  # dict.get() checks if the key exists,
                                        # it returns None if it is not in the dictionary 
            if idx_word: # it exists
                idx_word.update({filename: freq}) # add the filename to the index
                idx_word['df'] = (idx_word['df'] * N + 1) / N  # recalculate the DF
            else:                       # the word is not in the index
                index[word] = {filename : freq, 'df': 1 / N} # insert the word
    return index

###Querying and scoring
Querying is done by entering a string of words and then summing up the TF\*IDF score for each word in the query for each document in the collection.





In [7]:
def score(word, doc, index):
    '''
    Returns the TFxIDF score for a given word and index.
    '''
    if index.get(word): # again we use the get function to prevent keyerrors
        TF = index.get(word).get(doc, 0)  # same as index[word][doc]
        IDF = math.log(1/index.get(word).get('df', 1))  # same as index[word]['df']
        return TF * IDF
    return 0 # the word was not in the document so we return a score of 0


def query(index, qry):
    qry = nltk.tokenize.word_tokenize(qry.lower())
    # create a list of tuples (score sums, document)
    score_list = [(sum(score(word, doc, index) for word in qry), doc)
                   for doc in index['__files']]
    # sort the document in descending order based on the scores
    return sorted(score_list, reverse=True)

##Extrinsic
###PageRank

When calculating the relevance of a word to a document, you are using the intrinsic information of this and all other documents to do so (TF*IDF, Cosine Similarity).

You can also use the extrinsic information of a collection of documents (or webpages/social network nodes) to give each document an importance score. This is not dependent on a particular query and is deducted from the collective intelligence of a dataset/corpus. We will now use the example of webpages to elaborate on this.

When page A links to page B, page A basically vouches for/recommends page B. Thus, the amount of incoming links a page has can be used to calculate its general importance. This is called PageRank. Aside from the amount of incoming links a page has, PageRank also takes the importance of these incoming links into account.

However, at the very beginning of starting PageRanking the pages in your collection, no page has a PageRank yet. Thus, at this starting point (t = 0), we assume all pages have the same PageRank: 1 (one) divided by the amount of pages in the collection.

Example: all the pages in a collection of 5 pages don't have a PageRank yet. The score of each page is 1 / 5 = 0.2 at t = 0.

From this point on, calculating the PageRank of a particular page in a collection is an iterative process, as denoted by the following formula:


$$PR(i, t+1) = \frac{1 - d}{N} \ + \ d \ \cdot \ \sum_{j \in N(i)}\frac{PR(j, t)}{outdegree(j)}$$
PR(i, t+1) = \frac{1 - d}{N} \ + \ d \ \cdot \ \sum_{j \in N(i)}\frac{PR(j, t)}{outdegree(j)}

Let's take it slowly, one step at a time. To calculate the PageRank of page i:
First, we must account for a damping factor (d). This factor, set at 0.85, prevents the formula from not working with pages without outlinks.
We then calculate a prefix  $$\frac{1 - d}{N}$$ which can be seen as the probability of visiting a randomly chosen page. In this prefix, N is equal to the amount of pages in the collection.
When then take the damping factor d (remember, 0.85), and multiply it with…
The sum (for every incoming link j page i has) of…
The current PageRank of page j, divided by…
The outdegree of page j (how many other pages j links to and therefore 'gives' a little bit of its PageRank to)

After executing this formula over and over (remember, iterative process) for all pages until there's no change anymore in the calculated PageRank, we must normalize the then found PageRanks. This is done by dividing the PageRank of each page by the sum of all PageRanks.

This table shows the growth/evolution the PageRank process for the graph below. Each node (0 … 4) can be seen as a webpage on a tiny version of the internet (our collection), consisting of 5 webpages total.


|nodes | 0 | 1 | 2 | 3 | 4 | 
|-|---|---|---|---|---|
| inlinks | {1, 3} | {4} | {1} | {} | {0, 1, 2, 3} |
| outlinks | {0} | {0, 2, 4} | {4} | {0, 4} | {1} |
| t = 0 | ⅕ = 0.20 | ⅕ = 0.20 | ⅕ = 0.20 | ⅕ = 0.20 | ⅕ = 0.20 |
| t = 1 | 0.171666666 | 0.200000000 | 0.086666666 | 0.030000000 | 0.319000000 |
| t = 2 | 0.099416666 | 0.301150000 | 0.115325833 | 0.030000000 | 0.310606958 |
| t = 3 | 0.128075833 | 0.294015914 | 0.113304509 | 0.030000000 | 0.331227800 |
| t = 4 | 0.126054509 | 0.311543630 | 0.118270695 | 0.030000000 | 0.338697118 |
| t = 5 | 0.131020695 | 0.317892551 | 0.120069556 | 0.030000000 | 0.346246269 |
| Normalize 0.945229072 | 0.13 / 0.95 = 0.13861263 | 0.32 / 0.95 = 0.33631271 | 0.12 / 0.95 = 0.12702693 | 0.03 / 0.95 = 0.03173833 | 0.35 / 0.95 = 0.36630937 | 
Please remember that t = 5 is not enough for accurate PageRanking. Going as far as t = 20 would be better.

Here's some example calculations for PageRank:
$$PR(page \ 0, t1) = (\frac{1 - 0.85}{5}) \ + \  (0.85 \ \cdot \ (\frac{0.20}{3} \ + \ \frac{0.20}{2})) = 0.1717$$

$$PR(page \ 3, t1) = (\frac{1 - 0.85}{5}) \ + \  (0.85 \ \cdot \ (\frac{0}{0})) = 0.03$$

$$PR(page \ 4, t5) = (\frac{1 - 0.85}{5}) \ + \  (0.85 \ \cdot \ 
(\frac{0.126}{1} \ + \ \frac{0.311}{3} \ + \ \frac{0.119}{1} \ 
+ \ \frac{0.030}{2})) = 0.3462$$

When we implement this in Python, you'll see we'll get the same results:
from __future__ import division # always include this, solves ZeroDivision errors
import networkx as nx

H = nx.DiGraph() # directed NetworkX network
H.add_edges_from([(0, 4), (1, 0), (1, 2), (1, 4), (2, 4), (3, 0), (3, 4), (4, 1)]) # building up our collection/'internet'

nx.pagerank(H) # inbuilt PageRank function of NetworkX
```
>>>{0: 0.1390711045911938,
1: 0.33995484588171887,
2: 0.12632110459119378,
3: 0.030000000000000006,
4: 0.3646529449358938}
```
```
def pagerank(H, iters = 20, d = .85):
    N = len(H)
    prefix = (1 - d) / N

    # Two ways to calculate ranks at t = 0:
    ranks = {x: 1 / N for x in range(N + 1)}
    ranks = dict.fromkeys(H, 1 / N)
    
    # For each iteration... 
    for _ in range(iters):
        
        # We take each page/node
        for page in ranks:
            
            # We list its friends (in-links)
            friends = [f[0] for f in H.in_edges(page)]
            
            # We update its rank with...
            # The prefix + d multiplied by...
            # The sum of, for each friend of page,
            # It's current PageRank divided by its outdegree
            ranks[page] = prefix + d * \
            sum(ranks[f] / H.out_degree(f) for f in friends)

        
    # We're done, now let's normalize all PageRanks
    norm = sum(ranks.values())
    return {k: ranks[k] / norm for k in ranks}
```
```
pagerank(H, iters = 5)
>>>
{0: 0.13861263800372547,
 1: 0.33631271023979153,
 2: 0.1270269394649432,
 3: 0.03173833823033561,
 4: 0.36630937406120423}
```